In [ ]:
# ------------------ IMPORTS ------------------
import os
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
from tqdm import tqdm

# ------------------ CONFIG ------------------
HF_TOKEN = "hf_token"  # tu token de Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN

MODEL_LLM = "mistralai/Mistral-7B-v0.1"

DATA_PATH = "/home/jovyan/lrec_2026/data_10/migration/glossary_migration_info_semantica_es.csv"
OUTPUT_PATH = "/home/jovyan/lrec_2026/data_10/migration/migration_definitions_info_semántica_mistral_es.csv"

LIMIT_TERMS = 100

# ------------------ CARGA CSV ------------------
print("📥 Cargando CSV...")
df = pd.read_csv(DATA_PATH, sep=";", quotechar='"', engine="python", on_bad_lines="skip")
print(f"✅ CSV cargado con {len(df)} filas y columnas: {list(df.columns)}")

if "term" not in df.columns or "info_semantica" not in df.columns:
    raise ValueError("❌ El CSV debe tener las columnas 'term' y 'info_semantica'.")

# Limitar cantidad de términos
df = df.head(LIMIT_TERMS)

# ------------------ LIMPIEZA ------------------
def clean_html(text):
    if pd.isna(text):
        return ""
    return BeautifulSoup(str(text), "html.parser").get_text(separator=" ", strip=True)

df["info_semantica"] = df["info_semantica"].apply(clean_html).fillna("")
df["term"] = df["term"].astype(str).fillna("")

# ------------------ MODELO LLM ------------------
print("🔹 Cargando modelo Mistral 7B...")
llm_pipeline = pipeline(
    "text-generation",
    model=MODEL_LLM,
    tokenizer=MODEL_LLM,
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=256,
    temperature=0.5,
    repetition_penalty=1.1
)
print("✅ Modelo cargado correctamente")

# ------------------ GENERAR DEFINICIONES ------------------
def generate_definition(term, info_semantica):
    context = info_semantica.strip() or "No hay información semántica disponible."
    prompt = f"""Eres un experto en terminología.
Basándote únicamente en la siguiente información semántica, redacta una definición precisa y concisa en ESPAÑOL para el término: "{term}". 
NO GENERES DEFINICIONES EN INGLÉS. NO INCLUYAS SALTOS DE LÍNEA.

Información semántica:
{context}

Definición:
"""
    output = llm_pipeline(prompt)[0]["generated_text"]
    return output.split("Definición:")[-1].strip()

# ------------------ PROCESAMIENTO ------------------
definitions = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="🧠 Generando definiciones"):
    try:
        definition = generate_definition(row["term"], row["info_semantica"])
    except Exception as e:
        definition = f"ERROR: {e}"
    definitions.append(definition)

df["definition"] = definitions

df.to_csv(OUTPUT_PATH, sep=";", index=False)
print(f"\n✅ Archivo final guardado en: {OUTPUT_PATH}")
print(f"📘 Términos procesados: {len(df)}")
